IDS 561 - Assignment 3

Group Submission By:
Priyanka Shah
Rini Thomas
Vanshdeep Singh

In [0]:
# Mounting Gdrive

In [255]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Step 2. Install Spark

# Get Spark installer (check the path on spark.apache.org)

In [0]:
!wget -q http://apache.mirrors.pair.com/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz

# Check if the file is copied

In [258]:
!df -h 

Filesystem      Size  Used Avail Use% Mounted on
overlay         108G   29G   74G  29% /
tmpfs            64M     0   64M   0% /dev
tmpfs           6.4G     0  6.4G   0% /sys/fs/cgroup
tmpfs           6.4G   12K  6.4G   1% /var/colab
/dev/sda1       114G   31G   84G  27% /etc/hosts
shm             5.9G     0  5.9G   0% /dev/shm
tmpfs           6.4G     0  6.4G   0% /proc/acpi
tmpfs           6.4G     0  6.4G   0% /proc/scsi
tmpfs           6.4G     0  6.4G   0% /sys/firmware
drive           108G   38G   70G  36% /content/drive


In [259]:
!rm 

rm: missing operand
Try 'rm --help' for more information.


In [260]:
!ls

'My Drive'  'Shared drives'


# Untar the Spark installer

In [261]:
!tar -xvf spark-2.4.5-bin-hadoop2.7.tgz

tar: spark-2.4.5-bin-hadoop2.7.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


# Check the Spark folder after untar

In [262]:
!ls 

'My Drive'  'Shared drives'


# Install findspark - a python library to find Spark

In [0]:
!pip install -q findspark

# Step 3. Set environment variables
Set Java and Spark home based on the location where they are stored

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

# Step 4. Creat a local Spark session

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [266]:
import pandas as pd
import os

## Setting the working Directory
os.chdir("/content/drive/")
!ls

'My Drive'  'Shared drives'


In [267]:
## Reading the csv File
df = spark.read.csv("/content/drive/My Drive/Colab_Datasets/Amazon_Responded_Oct05.csv",header = True)  
df.show()                           

+--------------------+--------------------+----------------+-----------+-------------------+---------------------+--------------+-----------------+--------------+--------------------+-------------+-------------+--------------------+------------------+--------------------+--------------+--------------------+--------------+---------+-----------------------+-------------------------+-----------------------+-------------+---------+--------------------+
|              id_str|    tweet_created_at|user_screen_name|user_id_str|user_statuses_count|user_favourites_count|user_protected|user_listed_count|user_following|    user_description|user_location|user_verified|user_followers_count|user_friends_count|     user_created_at|tweet_language|               text_|favorite_count|favorited|in_reply_to_screen_name|in_reply_to_status_id_str|in_reply_to_user_id_str|retweet_count|retweeted|                text|
+--------------------+--------------------+----------------+-----------+-------------------+--

In [268]:
## Checking the columns in the dataset

df.columns

['id_str',
 'tweet_created_at',
 'user_screen_name',
 'user_id_str',
 'user_statuses_count',
 'user_favourites_count',
 'user_protected',
 'user_listed_count',
 'user_following',
 'user_description',
 'user_location',
 'user_verified',
 'user_followers_count',
 'user_friends_count',
 'user_created_at',
 'tweet_language',
 'text_',
 'favorite_count',
 'favorited',
 'in_reply_to_screen_name',
 'in_reply_to_status_id_str',
 'in_reply_to_user_id_str',
 'retweet_count',
 'retweeted',
 'text']

In [269]:
## No of rows in the data

print(df.count())

413247


In [270]:
## Selecting the 3 columns we require

df2 = df.select('tweet_created_at','user_screen_name','user_id_str')
df2.show()

+--------------------+----------------+-----------+
|    tweet_created_at|user_screen_name|user_id_str|
+--------------------+----------------+-----------+
|Tue Nov 01 01:57:...|     SeanEPanjab|  143515471|
|Tue Nov 01 02:39:...|      AmazonHelp|   85741735|
|Tue Nov 01 17:14:...|     SeanEPanjab|  143515471|
|Tue Nov 01 17:15:...|     SeanEPanjab|  143515471|
|Tue Nov 01 17:19:...|      AmazonHelp|   85741735|
|Tue Nov 01 17:25:...|      AmazonHelp|   85741735|
|Tue Nov 01 17:55:...|     SeanEPanjab|  143515471|
|Tue Nov 01 17:55:...|     SeanEPanjab|  143515471|
|Tue Nov 01 18:02:...|      AmazonHelp|   85741735|
|Tue Nov 01 03:51:...|   aakashwangnoo|   71457972|
|Tue Nov 01 03:59:...|      AmazonHelp|   85741735|
|Tue Nov 01 11:00:...|   aakashwangnoo|   71457972|
|Tue Nov 01 12:03:...|      AmazonHelp|   85741735|
|Tue Nov 01 12:03:...|      AmazonHelp|   85741735|
|Tue Nov 01 14:57:...|   aakashwangnoo|   71457972|
|Tue Nov 01 16:38:...|      AmazonHelp|   85741735|
|Tue Nov 01 

In [271]:
## Checking the data type of each variable

df2.dtypes

[('tweet_created_at', 'string'),
 ('user_screen_name', 'string'),
 ('user_id_str', 'string')]

In [272]:
type(df2)

pyspark.sql.dataframe.DataFrame

In [273]:
## Subsetting the tweet Date column to extract only the Day and Month

df3 = df2.withColumn('Created_date',df2.tweet_created_at[5:6])
df3.show()

+--------------------+----------------+-----------+------------+
|    tweet_created_at|user_screen_name|user_id_str|Created_date|
+--------------------+----------------+-----------+------------+
|Tue Nov 01 01:57:...|     SeanEPanjab|  143515471|      Nov 01|
|Tue Nov 01 02:39:...|      AmazonHelp|   85741735|      Nov 01|
|Tue Nov 01 17:14:...|     SeanEPanjab|  143515471|      Nov 01|
|Tue Nov 01 17:15:...|     SeanEPanjab|  143515471|      Nov 01|
|Tue Nov 01 17:19:...|      AmazonHelp|   85741735|      Nov 01|
|Tue Nov 01 17:25:...|      AmazonHelp|   85741735|      Nov 01|
|Tue Nov 01 17:55:...|     SeanEPanjab|  143515471|      Nov 01|
|Tue Nov 01 17:55:...|     SeanEPanjab|  143515471|      Nov 01|
|Tue Nov 01 18:02:...|      AmazonHelp|   85741735|      Nov 01|
|Tue Nov 01 03:51:...|   aakashwangnoo|   71457972|      Nov 01|
|Tue Nov 01 03:59:...|      AmazonHelp|   85741735|      Nov 01|
|Tue Nov 01 11:00:...|   aakashwangnoo|   71457972|      Nov 01|
|Tue Nov 01 12:03:...|   

In [274]:
df3.dtypes

[('tweet_created_at', 'string'),
 ('user_screen_name', 'string'),
 ('user_id_str', 'string'),
 ('Created_date', 'string')]

In [0]:
## Converting the Date column from String into Datetype

from pyspark.sql.types import DateType
from pyspark.sql.functions import col, unix_timestamp, to_date

df4 = df3.withColumn('Created_date2', 
                  to_date(unix_timestamp(col('Created_date'), format = 'MMM dd').cast("timestamp")))

# df4 = df3.withColumn('Created_date2', df3['Created_date'].cast(DateType()))

In [276]:
df4.dtypes

[('tweet_created_at', 'string'),
 ('user_screen_name', 'string'),
 ('user_id_str', 'string'),
 ('Created_date', 'string'),
 ('Created_date2', 'date')]

In [277]:
df4.show()

+--------------------+----------------+-----------+------------+-------------+
|    tweet_created_at|user_screen_name|user_id_str|Created_date|Created_date2|
+--------------------+----------------+-----------+------------+-------------+
|Tue Nov 01 01:57:...|     SeanEPanjab|  143515471|      Nov 01|   1970-11-01|
|Tue Nov 01 02:39:...|      AmazonHelp|   85741735|      Nov 01|   1970-11-01|
|Tue Nov 01 17:14:...|     SeanEPanjab|  143515471|      Nov 01|   1970-11-01|
|Tue Nov 01 17:15:...|     SeanEPanjab|  143515471|      Nov 01|   1970-11-01|
|Tue Nov 01 17:19:...|      AmazonHelp|   85741735|      Nov 01|   1970-11-01|
|Tue Nov 01 17:25:...|      AmazonHelp|   85741735|      Nov 01|   1970-11-01|
|Tue Nov 01 17:55:...|     SeanEPanjab|  143515471|      Nov 01|   1970-11-01|
|Tue Nov 01 17:55:...|     SeanEPanjab|  143515471|      Nov 01|   1970-11-01|
|Tue Nov 01 18:02:...|      AmazonHelp|   85741735|      Nov 01|   1970-11-01|
|Tue Nov 01 03:51:...|   aakashwangnoo|   71457972| 

In [0]:
# Task 1

In [279]:
## Counting the number of distinct dates for each User

from pyspark.sql import functions as func

df5 = df4.groupby('user_id_str').agg(func.countDistinct('Created_date2').alias("daycount"))
df6 = df5.orderBy(df5[1].desc())
df6.show(5)

+-----------+--------+
|user_id_str|daycount|
+-----------+--------+
|   85741735|     279|
|   54704557|      27|
|   15413409|      19|
|  441572163|      16|
|  362763757|      16|
+-----------+--------+
only showing top 5 rows



In [280]:
print(df6.count())

74134


In [281]:
print(df6.select('user_id_str').distinct().count())

74134


In [282]:
## Subsetting for those Users that have tweeted for atleast 5 days

df7 = df6.filter(df6.daycount > 4)
print(df7.count())

593


In [283]:
df7.show(5)

+-----------+--------+
|user_id_str|daycount|
+-----------+--------+
|   85741735|     279|
|   54704557|      27|
|   15413409|      19|
|  441572163|      16|
|  133804026|      16|
+-----------+--------+
only showing top 5 rows



In [284]:
## Creating the Daily Active Users Dataframe

daily_active_users = df7.join(df2, on=['user_id_str'], how='inner').select('user_screen_name','user_id_str').distinct()
daily_active_users.show()

+----------------+------------------+
|user_screen_name|       user_id_str|
+----------------+------------------+
|   Gentlemen_Sam|         441572163|
|       SkullyRox|          20391647|
|  sky_regenrated|         483059773|
|  whisperandmoan|         113516042|
|          MtnrMS|        3309102108|
|        remakoul|814372928695521280|
| roxyunderwood93|         295334669|
|        TCMuffin|          35591749|
|  ChaurasiaRohin|706032993794527232|
|     ssagardutta|        2991490598|
|        trallyus|          11702402|
|       basusagar|          42205044|
|      Whitjoseph|         148495262|
| Savvyy_Investor|758744557315993600|
|  tarunwadhwa198|         130872938|
|   Shadowslightz|        1282432266|
|         i_opner|        4189195819|
|   LadySlaughter|          15195593|
|    brian_riback|         424400406|
|          fadari|          25689668|
+----------------+------------------+
only showing top 20 rows



In [285]:
## No of unique Daily Users

print(daily_active_users.count())

593


In [0]:
## Task 2

In [0]:
## Reading the experiment text file

text2 = spark.read.text("/content/drive/My Drive/Colab_Datasets/experiment.txt")

In [288]:
text2.show(3)

+---------+
|    value|
+---------+
|143515471|
| 85741735|
| 71457972|
+---------+
only showing top 3 rows



In [289]:
## Checking the number of IDs in the file

print(text2.select('value').distinct().count())

5000


In [290]:
print(text2.count())

5000


In [291]:
## Combining the Daily Users data and Experiment data

active_users = text2.join(daily_active_users, text2.value == daily_active_users.user_id_str,how='left')
active_users.show()

+----------+----------------+-----------+
|     value|user_screen_name|user_id_str|
+----------+----------------+-----------+
| 106799492|            null|       null|
|2610379644|            null|       null|
|  62364020|            null|       null|
| 123138418|            null|       null|
|2908108256|            null|       null|
| 143515471|            null|       null|
|  97424433|            null|       null|
|2706101936|            null|       null|
|1510968974|            null|       null|
| 110354554| praveen_pandey_|  110354554|
|2502172122|            null|       null|
| 243716471|            null|       null|
|  59156981|            null|       null|
|  71457972|            null|       null|
| 257376764|            null|       null|
|   5654472|            null|       null|
|  85741735|      AmazonHelp|   85741735|
| 370711133|            null|       null|
| 145579921|            null|       null|
| 902137872|     mybharatraj|  902137872|
+----------+----------------+-----

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *


In [293]:
active_users.show(5)

+----------+----------------+-----------+
|     value|user_screen_name|user_id_str|
+----------+----------------+-----------+
| 106799492|            null|       null|
|2908108256|            null|       null|
| 143515471|            null|       null|
|  59156981|            null|       null|
|  71457972|            null|       null|
+----------+----------------+-----------+
only showing top 5 rows



In [294]:
active_users.columns

['value', 'user_screen_name', 'user_id_str']

In [0]:
## Creating the Experiment User Dataframe

experiment_user = active_users.withColumn('Whether_active' ,when(col("user_screen_name").isNull(), "NO").otherwise("YES")).select('value','Whether_active')

In [296]:
experiment_user.show(5)

+----------+--------------+
|     value|Whether_active|
+----------+--------------+
|  11798342|            NO|
|1210875679|            NO|
| 128257538|            NO|
| 137088213|            NO|
| 142202059|            NO|
+----------+--------------+
only showing top 5 rows



In [297]:
## Getting the counts of the Whether Active Variable

active_counts = experiment_user.groupBy('Whether_active').count().withColumnRenamed("count", "active_cnt")
active_counts.show()

+--------------+----------+
|Whether_active|active_cnt|
+--------------+----------+
|           YES|       121|
|            NO|      4879|
+--------------+----------+



In [298]:
## Getting the percentage of the counts

total_count = experiment_user.count()

percent_output = active_counts.withColumn('percentage', active_counts.active_cnt/float(total_count))
percent_output.show()

+--------------+----------+----------+
|Whether_active|active_cnt|percentage|
+--------------+----------+----------+
|           YES|       121|    0.0242|
|            NO|      4879|    0.9758|
+--------------+----------+----------+



In [0]:
## Task 3

In [300]:
daily_active_users.show(2)

+----------------+-----------+
|user_screen_name|user_id_str|
+----------------+-----------+
|   Gentlemen_Sam|  441572163|
|       SkullyRox|   20391647|
+----------------+-----------+
only showing top 2 rows



In [301]:
experiment_user.show(2)

+----------+--------------+
|     value|Whether_active|
+----------+--------------+
|  11798342|            NO|
|1210875679|            NO|
+----------+--------------+
only showing top 2 rows



In [302]:
## Joining the Users of Daily Active and Experiment

Cross_users = experiment_user.join(daily_active_users, experiment_user.value == daily_active_users.user_id_str,how='outer')
Cross_users.show()


+----------+--------------+----------------+-----------+
|     value|Whether_active|user_screen_name|user_id_str|
+----------+--------------+----------------+-----------+
|  11798342|            NO|            null|       null|
|1210875679|            NO|            null|       null|
| 128257538|            NO|            null|       null|
| 137088213|            NO|            null|       null|
| 142202059|            NO|            null|       null|
|  14291504|            NO|            null|       null|
|  16279527|            NO|            null|       null|
| 163148814|            NO|            null|       null|
|      null|          null|        JoeMedal|   19191538|
|  20110100|            NO|            null|       null|
| 211972025|            NO|            null|       null|
|2176836186|            NO|            null|       null|
| 234641258|            NO|            null|       null|
|2372082613|            NO|            null|       null|
|  26915435|            NO|    

In [303]:
## Creating a dummy variable to check which Users are in the Daily Active as well as Experiment Data

Cross_users = Cross_users.withColumn('Experiment_user' ,when(col("Whether_active").isNull(), 0).otherwise(1))
Cross_users = Cross_users.withColumn('Daily_user' ,when(col("user_id_str").isNull(), 0).otherwise(1))
Cross_users = Cross_users.withColumn('Daily_and_Experiment_user' ,when((col("Experiment_user")==1) & (col("Daily_user")==1), 1).otherwise(0))
Cross_users = Cross_users.withColumn('New_ID' ,when(col("value").isNull(), col("user_id_str")).otherwise(col("value")))

Cross_users.show()

+----------+--------------+----------------+-----------+---------------+----------+-------------------------+----------+
|     value|Whether_active|user_screen_name|user_id_str|Experiment_user|Daily_user|Daily_and_Experiment_user|    New_ID|
+----------+--------------+----------------+-----------+---------------+----------+-------------------------+----------+
|  11798342|            NO|            null|       null|              1|         0|                        0|  11798342|
|1210875679|            NO|            null|       null|              1|         0|                        0|1210875679|
| 128257538|            NO|            null|       null|              1|         0|                        0| 128257538|
| 137088213|            NO|            null|       null|              1|         0|                        0| 137088213|
| 142202059|            NO|            null|       null|              1|         0|                        0| 142202059|
|  14291504|            NO|     

In [304]:
print(Cross_users.count())

5472


In [305]:
Cross_users.groupBy('Daily_and_Experiment_user').count().show()

+-------------------------+-----+
|Daily_and_Experiment_user|count|
+-------------------------+-----+
|                        1|  121|
|                        0| 5351|
+-------------------------+-----+



In [306]:
## Subsetting only those User IDs which satisfy both criteria

Both_users = Cross_users.filter(Cross_users.Daily_and_Experiment_user ==1).select('New_ID')
Both_users.count()

121

In [307]:
Both_users.show(5)

+----------+
|    New_ID|
+----------+
|  18435372|
|1387267123|
| 267381861|
| 885336408|
| 213060183|
+----------+
only showing top 5 rows



In [308]:
df.show(5)

+--------------------+--------------------+----------------+-----------+-------------------+---------------------+--------------+-----------------+--------------+--------------------+-------------+-------------+--------------------+------------------+--------------------+--------------+--------------------+--------------+---------+-----------------------+-------------------------+-----------------------+-------------+---------+--------------------+
|              id_str|    tweet_created_at|user_screen_name|user_id_str|user_statuses_count|user_favourites_count|user_protected|user_listed_count|user_following|    user_description|user_location|user_verified|user_followers_count|user_friends_count|     user_created_at|tweet_language|               text_|favorite_count|favorited|in_reply_to_screen_name|in_reply_to_status_id_str|in_reply_to_user_id_str|retweet_count|retweeted|                text|
+--------------------+--------------------+----------------+-----------+-------------------+--

In [309]:
## Combining the new set of users with the Original Amazon File

New_users = Both_users.join(df, Both_users.New_ID == df.user_id_str,how='left').drop('New_ID')
New_users.count()


172767

In [310]:
df.count()

413247

In [0]:
## Exporting the file to a csv

New_users.toPandas().to_csv('/content/drive/My Drive/Colab_Datasets/Amazon_New.csv')